In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import glob

In [ ]:
pngs = glob.glob('sd02/data/*/*/*.png')
pngs.extend(glob.glob('sd06/data/*/*/*.png'))
tags = [open(png[:-4] + '.fmt').readline().strip() for png in pngs]
len(pngs), len(tags)

In [ ]:
df = pd.DataFrame(dict(fname=pngs, label=tags))
df.head()

In [ ]:
df['label'].value_counts(normalize=True)

In [ ]:
from fastai.vision import *
from fastai.metrics import error_rate

In [ ]:
img_size = 224
bs = 64

In [ ]:
data = ImageDataBunch.from_df('.', df, fn_col='fname', label_col='label', \
                              ds_tfms=get_transforms(), size=img_size, bs=bs).normalize(imagenet_stats)

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(4, max_lr=0.01)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))